<a href="https://colab.research.google.com/github/Viny2030/sklearn/blob/main/ensemble_hyperparameters_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter tuning

In the previous section, we did not discuss the hyperparameters of random
forest and histogram gradient-boosting. This notebook gives crucial
information regarding how to set them.

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">For the sake of clarity, no nested cross-validation is used to estimate the
variability of the testing error. We are only showing the effect of the
parameters on the validation set.</p>
</div>

We start by loading the california housing dataset.

# Ajuste de hiperparámetros
En la sección anterior, no analizamos los hiperparámetros del bosque aleatorio ni la potenciación del gradiente del histograma. Este cuaderno proporciona información crucial sobre cómo configurarlos.

# ¡Precaución!

Para mayor claridad, no se utiliza ninguna validación cruzada anidada para estimar la variabilidad del error de prueba. Solo mostramos el efecto de los parámetros en el conjunto de validación.

Comenzamos cargando el conjunto de datos de vivienda de California.
Enviar comentarios

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0
)

## Random forest

The main parameter to select in random forest is the `n_estimators` parameter.
In general, the more trees in the forest, the better the generalization
performance would be. However, adding trees slows down the fitting and prediction
time. The goal is to balance computing time and generalization performance
when setting the number of estimators. Here, we fix `n_estimators=100`, which
is already the default value.

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Tuning the <tt class="docutils literal">n_estimators</tt> for random forests generally result in a waste of
computer power. We just need to ensure that it is large enough so that doubling
its value does not lead to a significant improvement of the validation error.</p>
</div>

Instead, we can tune the hyperparameter `max_features`, which controls the
size of the random subset of features to consider when looking for the best
split when growing the trees: smaller values for `max_features` lead to
more random trees with hopefully more uncorrelated prediction errors. However
if `max_features` is too small, predictions can be too random, even after
averaging with the trees in the ensemble.

If `max_features` is set to `None`, then this is equivalent to setting
`max_features=n_features` which means that the only source of randomness in
the random forest is the bagging procedure.

# **Sección nueva**
# Bosque aleatorio
El parámetro principal que se debe seleccionar en un bosque aleatorio es el parámetro n_estimators. En general, cuantos más árboles haya en el bosque, mejor será el rendimiento de generalización. Sin embargo, agregar árboles ralentiza el tiempo de ajuste y predicción. El objetivo es equilibrar el tiempo de cálculo y el rendimiento de generalización al establecer la cantidad de estimadores. Aquí, fijamos n_estimators=100, que ya es el valor predeterminado.

¡Precaución!

Ajustar los n_estimators para bosques aleatorios generalmente resulta en un desperdicio de potencia de cómputo. Solo debemos asegurarnos de que sea lo suficientemente grande para que duplicar su valor no conduzca a una mejora significativa del error de validación.

En cambio, podemos ajustar el hiperparámetro max_features, que controla el tamaño del subconjunto aleatorio de características a considerar cuando se busca la mejor división al hacer crecer los árboles: valores más pequeños para max_features conducen a más árboles aleatorios con, con suerte, más errores de predicción no correlacionados. Sin embargo, si max_features es demasiado pequeño, las predicciones pueden ser demasiado aleatorias, incluso después de promediar con los árboles del conjunto.

Si max_features se establece en None, esto es equivalente a establecer max_features=n_features, lo que significa que la única fuente de aleatoriedad en el bosque aleatorio es el procedimiento de bagging.
Enviar comentarios


In [2]:
print(f"In this case, n_features={len(data.columns)}")

In this case, n_features=8


We can also tune the different parameters that control the depth of each tree
in the forest. Two parameters are important for this: `max_depth` and
`max_leaf_nodes`. They differ in the way they control the tree structure.
Indeed, `max_depth` enforces growing symmetric trees, while `max_leaf_nodes`
does not impose such constraint. If `max_leaf_nodes=None` then the number of
leaf nodes is unlimited.

The hyperparameter `min_samples_leaf` controls the minimum number of samples
required to be at a leaf node. This means that a split point (at any depth) is
only done if it leaves at least `min_samples_leaf` training samples in each of
the left and right branches. A small value for `min_samples_leaf` means that
some samples can become isolated when a tree is deep, promoting overfitting. A
large value would prevent deep trees, which can lead to underfitting.

Be aware that with random forest, trees are expected to be deep since we are
seeking to overfit each tree on each bootstrap sample. Overfitting is
mitigated when combining the trees altogether, whereas assembling underfitted
trees (i.e. shallow trees) might also lead to an underfitted forest.

También podemos ajustar los diferentes parámetros que controlan la profundidad de cada árbol en el bosque. Dos parámetros son importantes para esto: max_depth y max_leaf_nodes. Se diferencian en la forma en que controlan la estructura del árbol. De hecho, max_depth impone árboles simétricos en crecimiento, mientras que max_leaf_nodes no impone dicha restricción. Si max_leaf_nodes=None, entonces la cantidad de nodos de hoja es ilimitada.

El hiperparámetro min_samples_leaf controla la cantidad mínima de muestras que se requiere que estén en un nodo de hoja. Esto significa que un punto de división (a cualquier profundidad) solo se realiza si deja al menos min_samples_leaf muestras de entrenamiento en cada una de las ramas izquierda y derecha. Un valor pequeño para min_samples_leaf significa que algunas muestras pueden quedar aisladas cuando un árbol es profundo, lo que promueve el sobreajuste. Un valor alto evitaría árboles profundos, lo que puede llevar a un subajuste.

Tenga en cuenta que con un bosque aleatorio, se espera que los árboles sean profundos ya que buscamos sobreajustar cada árbol en cada muestra de arranque. El sobreajuste se mitiga al combinar los árboles en su totalidad, mientras que ensamblar árboles subadaptados (es decir, árboles poco profundos) también podría generar un bosque subadaptado.


In [3]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

param_distributions = {
    "max_features": [1, 2, 3, 5, None],
    "max_leaf_nodes": [10, 100, 1000, None],
    "min_samples_leaf": [1, 2, 5, 10, 20, 50, 100],
}
search_cv = RandomizedSearchCV(
    RandomForestRegressor(n_jobs=2),
    param_distributions=param_distributions,
    scoring="neg_mean_absolute_error",
    n_iter=10,
    random_state=0,
    n_jobs=2,
)
search_cv.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_distributions.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

,param_max_features,param_max_leaf_nodes,param_min_samples_leaf,mean_test_error,std_test_error
3,2,None,2,33.981602,0.475605
0,2,1000,10,36.862746,0.583592
7,None,None,20,37.382869,0.431934
4,5,100,2,40.139328,0.512141
8,None,100,10,40.428765,0.517156
6,None,1000,50,40.845814,0.582102
9,1,100,2,49.785084,1.009107
2,1,100,1,50.237545,0.609356
5,1,None,100,54.386208,0.780751
1,3,10,10,54.838013,0.822813


We can observe in our search that we are required to have a large number of
`max_leaf_nodes` and thus deep trees. This parameter seems particularly
impactful with respect to the other tuning parameters, but large values of
`min_samples_leaf` seem to reduce the performance of the model.

In practice, more iterations of random search would be necessary to precisely
assert the role of each parameters. Using `n_iter=10` is good enough to
quickly inspect the hyperparameter combinations that yield models that work
well enough without spending too much computational resources. Feel free to
try more interations on your own.

Once the `RandomizedSearchCV` has found the best set of hyperparameters, it
uses them to refit the model using the full training set. To estimate the
generalization performance of the best model it suffices to call `.score` on
the unseen data.

Podemos observar en nuestra búsqueda que se requiere que tengamos una gran cantidad de max_leaf_nodes y, por lo tanto, árboles profundos. Este parámetro parece tener un impacto particular con respecto a los otros parámetros de ajuste, pero los valores grandes de min_samples_leaf parecen reducir el rendimiento del modelo.

En la práctica, serían necesarias más iteraciones de búsqueda aleatoria para afirmar con precisión el papel de cada parámetro. El uso de n_iter=10 es suficiente para inspeccionar rápidamente las combinaciones de hiperparámetros que producen modelos que funcionan lo suficientemente bien sin gastar demasiados recursos computacionales. No dude en probar más interacciones por su cuenta.

Una vez que RandomizedSearchCV ha encontrado el mejor conjunto de hiperparámetros, los utiliza para reajustar el modelo utilizando el conjunto de entrenamiento completo. Para estimar el rendimiento de generalización del mejor modelo, basta con llamar a .score en los datos no vistos.


In [4]:
error = -search_cv.score(data_test, target_test)
print(
    f"On average, our random forest regressor makes an error of {error:.2f} k$"
)

On average, our random forest regressor makes an error of 33.67 k$


## Histogram gradient-boosting decision trees

For gradient-boosting, hyperparameters are coupled, so we cannot set them
one after the other anymore. The important hyperparameters are `max_iter`,
`learning_rate`, and `max_depth` or `max_leaf_nodes` (as previously discussed
random forest).

Let's first discuss `max_iter` which, similarly to the `n_estimators`
hyperparameter in random forests, controls the number of trees in the
estimator. The difference is that the actual number of trees trained by the
model is not entirely set by the user, but depends also on the stopping
criteria: the number of trees can be lower than `max_iter` if adding a new
tree does not improve the model enough. We will give more details on this in
the next exercise.

The depth of the trees is controlled by `max_depth` (or `max_leaf_nodes`). We
saw in the section on gradient-boosting that boosting algorithms fit the error
of the previous tree in the ensemble. Thus, fitting fully grown trees would be
detrimental. Indeed, the first tree of the ensemble would perfectly fit
(overfit) the data and thus no subsequent tree would be required, since there
would be no residuals. Therefore, the tree used in gradient-boosting should
have a low depth, typically between 3 to 8 levels, or few leaves ($2^3=8$ to
$2^8=256$). Having very weak learners at each step helps reducing overfitting.

With this consideration in mind, the deeper the trees, the faster the
residuals are corrected and then less learners are required. Therefore,
it can be beneficial to increase `max_iter` if `max_depth` is low.

Finally, we have overlooked the impact of the `learning_rate` parameter until
now. When fitting the residuals, we would like the tree to try to correct all
possible errors or only a fraction of them. The learning-rate allows you to
control this behaviour. A small learning-rate value would only correct the
residuals of very few samples. If a large learning-rate is set (e.g., 1), we
would fit the residuals of all samples. So, with a very low learning-rate, we
would need more estimators to correct the overall error. However, a too large
learning-rate tends to obtain an overfitted ensemble, similar to having very
deep trees.

Árboles de decisión de potenciación de gradiente de histograma
Para la potenciación de gradiente, los hiperparámetros están acoplados, por lo que ya no podemos establecerlos uno después del otro. Los hiperparámetros importantes son max_iter, learning_rate y max_depth o max_leaf_nodes (como se explicó anteriormente en el bosque aleatorio).

Primero analicemos max_iter que, de manera similar al hiperparámetro n_estimators en los bosques aleatorios, controla la cantidad de árboles en el estimador. La diferencia es que la cantidad real de árboles entrenados por el modelo no está completamente establecida por el usuario, sino que también depende de los criterios de detención: la cantidad de árboles puede ser menor que max_iter si agregar un nuevo árbol no mejora el modelo lo suficiente. Daremos más detalles sobre esto en el próximo ejercicio.

La profundidad de los árboles está controlada por max_depth (o max_leaf_nodes). Vimos en la sección sobre potenciación de gradiente que los algoritmos de potenciación se ajustan al error del árbol anterior en el conjunto. Por lo tanto, ajustar árboles completamente desarrollados sería perjudicial. De hecho, el primer árbol del conjunto se ajustaría perfectamente (sobreajustaría) a los datos y, por lo tanto, no se necesitaría ningún árbol posterior, ya que no habría residuos. Por lo tanto, el árbol utilizado en el aumento de gradiente debe tener una profundidad baja, normalmente entre 3 y 8 niveles, o pocas hojas (23=8 a 28=256). Tener aprendices muy débiles en cada paso ayuda a reducir el sobreajuste.

Con esta consideración en mente, cuanto más profundos sean los árboles, más rápido se corrigen los residuos y, por lo tanto, se requieren menos aprendices. Por lo tanto, puede ser beneficioso aumentar max_iter si max_depth es bajo.

Finalmente, hemos pasado por alto el impacto del parámetro learning_rate hasta ahora. Al ajustar los residuos, nos gustaría que el árbol intentara corregir todos los errores posibles o solo una fracción de ellos. La tasa de aprendizaje le permite controlar este comportamiento. Un valor pequeño de tasa de aprendizaje solo corregiría los residuos de muy pocas muestras. Si se establece una tasa de aprendizaje grande (por ejemplo, 1), ajustaríamos los residuos de todas las muestras. Por lo tanto, con una tasa de aprendizaje muy baja, necesitaríamos más estimadores para corregir el error general. Sin embargo, una tasa de aprendizaje demasiado alta tiende a generar un conjunto sobreajustado, similar a tener árboles muy profundos.


In [5]:
from scipy.stats import loguniform
from sklearn.ensemble import HistGradientBoostingRegressor

param_distributions = {
    "max_iter": [3, 10, 30, 100, 300, 1000],
    "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
    "learning_rate": loguniform(0.01, 1),
}
search_cv = RandomizedSearchCV(
    HistGradientBoostingRegressor(),
    param_distributions=param_distributions,
    scoring="neg_mean_absolute_error",
    n_iter=20,
    random_state=0,
    n_jobs=2,
)
search_cv.fit(data_train, target_train)

columns = [f"param_{name}" for name in param_distributions.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

,param_max_iter,param_max_leaf_nodes,param_learning_rate,mean_test_error,std_test_error
14,300,100,0.018640,30.894497,0.410389
6,300,20,0.047293,31.926092,0.260262
2,30,50,0.176656,32.451568,0.315783
13,300,10,0.297739,32.716226,0.484129
9,100,20,0.083745,33.148112,0.259702
19,100,10,0.215543,33.365411,0.433219
12,100,20,0.067503,33.563570,0.404289
16,300,5,0.059290,35.937264,0.407441
1,100,5,0.160519,36.242601,0.328834
0,1000,2,0.125207,41.035160,0.961632



<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Here, we tune <tt class="docutils literal">max_iter</tt> but be aware that it is better to set <tt class="docutils literal">max_iter</tt> to a
fixed, large enough value and use parameters linked to <tt class="docutils literal">early_stopping</tt> as we
will do in Exercise M6.04.</p>
</div>

In this search, we observe that for the best ranked models, having a
smaller `learning_rate`, requires more trees or a larger number of leaves
for each tree. However, it is particularly difficult to draw more detailed
conclusions since the best value of each hyperparameter depends on the other
hyperparameter values.

We can now estimate the generalization performance of the best model using the
test set.

¡Precaución!

Aquí, ajustamos max_iter, pero tenga en cuenta que es mejor establecer max_iter en un valor fijo lo suficientemente grande y usar parámetros vinculados a early_stopping como lo haremos en el Ejercicio M6.04.

En esta búsqueda, observamos que para los modelos mejor clasificados, tener una tasa de aprendizaje menor requiere más árboles o una mayor cantidad de hojas para cada árbol. Sin embargo, es particularmente difícil sacar conclusiones más detalladas ya que el mejor valor de cada hiperparámetro depende de los otros valores de hiperparámetro.

Ahora podemos estimar el rendimiento de generalización del mejor modelo utilizando el conjunto de prueba.


In [6]:
error = -search_cv.score(data_test, target_test)
print(f"On average, our HGBT regressor makes an error of {error:.2f} k$")

On average, our HGBT regressor makes an error of 30.44 k$


The mean test score in the held-out test set is slightly better than the score
of the best model. The reason is that the final model is refitted on the whole
training set and therefore, on more data than the cross-validated models of
the grid search procedure.

We summarize these details in the following table:

| **Bagging & Random Forests**                     | **Boosting**                                        |
|--------------------------------------------------|-----------------------------------------------------|
| fit trees **independently**                      | fit trees **sequentially**                          |
| each **deep tree overfits**                      | each **shallow tree underfits**                     |
| averaging the tree predictions **reduces overfitting** | sequentially adding trees **reduces underfitting** |
| generalization improves with the number of trees | too many trees may cause overfitting                |
| does not have a `learning_rate` parameter        | fitting the residuals is controlled by the `learning_rate` |

La puntuación media de la prueba en el conjunto de prueba retenido es ligeramente mejor que la puntuación del mejor modelo. La razón es que el modelo final se reajusta en todo el conjunto de entrenamiento y, por lo tanto, en más datos que los modelos validados de forma cruzada del procedimiento de búsqueda en cuadrícula.

Resumimos estos detalles en la siguiente tabla:

Bagging y Random Forests Boosting
Los árboles se ajustan de forma independiente. Los árboles se ajustan de forma secuencial.
Cada árbol profundo se sobreajusta. Cada árbol superficial se subajusta.
El promedio de las predicciones de los árboles reduce el sobreajuste. La adición de árboles de forma secuencial reduce el subajuste.
La generalización mejora con la cantidad de árboles. Demasiados árboles pueden causar sobreajuste.
No tiene un parámetro learning_rate. El ajuste de los residuos está controlado por learning_rate.
